In [11]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch import nn,optim
from torch.nn import functional as F 
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
traindata = np.load('2ddata.npy')
trainlabel = np.load('2dlabel.npy')

In [13]:
trainlabel

array([ 43.52606322, -13.99713994,   9.49569717, ...,  -7.8036471 ,
       -25.3031973 ,  -5.26887027])

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(traindata, trainlabel, test_size = 0.2, random_state = 1)
X_train, Y_train = torch.FloatTensor(X_train), torch.FloatTensor(Y_train)
X_test, Y_test = torch.FloatTensor(X_test), torch.FloatTensor(Y_test)

In [15]:
batchsize = 16
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)

#dataloader 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batchsize, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batchsize, shuffle = False)

In [16]:
X_train.shape 

torch.Size([8000, 256, 256])

In [17]:
class CNN_16d2(nn.Module):
    def __init__(self):
        super(CNN_16d2,self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 2,  stride=2)
        self.conv2 = nn.Conv2d(4, 16, 2, stride=2)
        self.conv3 = nn.Conv2d(16, 64, 2, stride=2)
        self.conv4 = nn.Conv2d(64, 256, 2, stride=2)
        self.conv5 = nn.Conv2d(256, 640, 2, stride=2)
        self.conv6 = nn.Conv2d(640, 160, 2, stride=2)
        self.conv7 = nn.Conv2d(160, 40, 2, stride=2)
        self.conv8 = nn.Conv2d(40, 10, 2, stride=2)


        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(10, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 20)
        self.fc4 = nn.Linear(20, 1)
    
    def forward(self,x):
        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = self.conv3(x)
        #print(x.shape)
        x = self.conv4(x)  
        #print(x.shape)
        x = self.conv5(x)  
        #print(x.shape)
        x = self.conv6(x)  
        #print(x.shape)
        x = self.conv7(x)  
        #print(x.shape)
        x = self.conv8(x)  
        #print(x.shape)
        x = self.flat(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [18]:
model = CNN_16d2()
if torch.cuda.is_available():
    print("gpu_available")
    model.cuda()

gpu_available


In [19]:
optimizer = optim.Adam(model.parameters(), lr =1)
criteon= nn.MSELoss()
EPOCH = 100
loss_list = []
for epoch in range(EPOCH):
    model.train()
    sum_loss = 0 
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        #data, target = Variable(data), Variable(target)

        data = torch.reshape(data,(batchsize, 1, 256,256))
        target = torch.reshape(target, (batchsize, 1))
        optimizer.zero_grad()
        
        
        data=data.to(torch.float32)
        target=target.to(torch.float32)
        output = model(data)

        
        #print(data)
        #print(target)
        #print("The output is:")
        #print(output)
        
        loss = criteon(output, target)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        if batch_idx % 100 == 99:
            print("train epoch {} loss : {:.3f} ^e6".format(epoch, sum_loss/100000000))
            loss_list.append(sum_loss/100)
            sum_loss = 0      

train epoch 0 loss : 783028705480066203648.000 ^e6
train epoch 0 loss : 803770231157373140992.000 ^e6
train epoch 0 loss : 774009698648911118336.000 ^e6
train epoch 0 loss : 786005982303446761472.000 ^e6
train epoch 0 loss : 784792664683003117568.000 ^e6
train epoch 1 loss : 789721949769887121408.000 ^e6
train epoch 1 loss : 788722601836912443392.000 ^e6
train epoch 1 loss : 791562288454531022848.000 ^e6
train epoch 1 loss : 778458183371039309824.000 ^e6
train epoch 1 loss : 788637168324279271424.000 ^e6
train epoch 2 loss : 780100709502413438976.000 ^e6
train epoch 2 loss : 790184612557999833088.000 ^e6
train epoch 2 loss : 790214091930769162240.000 ^e6
train epoch 2 loss : 794678945555718602752.000 ^e6
train epoch 2 loss : 779456620034856714240.000 ^e6
train epoch 3 loss : 774286773172116783104.000 ^e6
train epoch 3 loss : 796129810016166871040.000 ^e6
train epoch 3 loss : 798250554337061240832.000 ^e6
train epoch 3 loss : 782579238016924254208.000 ^e6
train epoch 3 loss : 7810332352

KeyboardInterrupt: 

NameError: name 'label' is not defined

In [ ]:
trainlabel

array([ 5.13073777e+05, -1.19680497e+04, -3.54062587e+03, ...,
        4.05250728e+02, -9.34886412e+03,  2.26318108e+04])